In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


import os
import random
import sys
import numpy as np
import pandas as pd

from ceb_gspan_mining.config import parser
from ceb_gspan_mining.ceb_gspan import gSpan

# 初始化仿真数据

In [2]:
# 首先分离数据

# vertexes: need_id,feature_black,label_black
# edges：src_id,dst_id,edge_type


def generate_graph(graphnum):
    graph = []
    vertexes = []
    edges = []
    # graph
    for g_id in range(graphnum):
        #vertex
        rand_v = 10000# int(random.random()*100)
        if rand_v<=10:continue
        for v in range(rand_v):
            graph.append(str(g_id)+str(0))
            vertexes.append(str(g_id)+str(v))
        
        rand_e = 100000 # int(random.random()*rand_v*rand_v)
        for e in range(rand_e):
            src = int(random.random()*rand_v)
            dst = int(random.random()*rand_v)
            if src!=dst:
                edges.append([g_id,str(src),str(dst)])

    df_vertex = pd.DataFrame({"graph_id":graph, "id":vertexes})
    df_vertex["node_label_0"] = np.random.randint(0,2,size=(len(vertexes),1))
    df_vertex["node_label_1"] = np.random.randint(0,2,size=(len(vertexes),1))
    df_edge = pd.DataFrame(np.array(edges),columns=["group_id",'src_id', 'dst_id'])
    df_edge["edge_type"] = np.random.randint(0,6,size=(len(edges),1))

    return df_vertex, df_edge

df_vertex, df_edge  = generate_graph(1)
df_vertex.to_csv("data/vertexes.csv")
df_edge[["group_id",'src_id', 'dst_id','edge_type']].to_csv("data/edges.csv")

In [3]:
top1K_neighes  = pd.DataFrame({"origin_id", "dst_id"})
top1K_neighes  = pd.DataFrame({"origin_id", "dst_id"})
top1K_neighes  = pd.DataFrame({"origin_id", "dst_id"})


In [4]:

# df_vertex = df_vertex[["id","node_label_0","node_label_1"]]
# df_edges = df_edge[['src_id', 'dst_id','edge_type']]

df_edge

,group_id,src_id,dst_id,edge_type
0,0,5471,2119,1
1,0,3737,214,2
2,0,2876,8057,3
3,0,4754,421,1
4,0,3085,957,1
...,...,...,...,...
99989,0,1146,7336,0
99990,0,706,6633,1
99991,0,8383,2898,5
99992,0,2265,9870,4


In [5]:
need_vertexes = df_vertex[df_vertex["node_label_1"]==1][["id"]][0:100]
need_vertexes

,id
4,04
7,07
8,08
9,09
11,011
...,...
203,0203
205,0205
206,0206
207,0207


In [6]:

# graphnum =1
# def generate_graph(graphnum):
#     graph = []
#     vertexes = []
#     edges = []
#     # graph
#     for g_id in range(graphnum):
#         #vertex
#         rand_v = int(random.random()*100)
#         if rand_v<=10:continue
#         for v in range(rand_v):
#             graph.append(str(g_id)+str(0))
#             vertexes.append(str(g_id)+str(v))
        
#         rand_e = int(random.random()*rand_v*rand_v)
#         for e in range(rand_e):
#             src = int(random.random()*rand_v)
#             dst = int(random.random()*rand_v)
#             if src!=dst:
#                 edges.append([g_id,str(g_id)+str(src),str(g_id)+str(dst)])

#     df_vertex = pd.DataFrame({"graph_id":graph, "id":vertexes})
#     df_vertex["node_label_0"] = np.random.randint(0,2,size=(len(vertexes),1))
#     df_vertex["node_label_1"] = np.random.randint(0,2,size=(len(vertexes),1))
#     df_edge = pd.DataFrame(np.array(edges),columns=["group_id",'src', 'dst'])
#     df_edge["edge_type"] = np.random.randint(0,6,size=(len(edges),1))

#     return df_vertex, df_edge

# df_vertex, df_edge  = generate_graph(50)
# df_vertex.to_csv("data/vertexes.csv")
# df_edge[['src', 'dst','edge_type']].to_csv("data/edges.csv")

In [7]:
def main(FLAGS=None):
    """Run Ceb gSpan."""

    if FLAGS is None:
        FLAGS, _ = parser.parse_known_args(args=["graphdata/graph.data.5"])

    if not os.path.exists(FLAGS.database_file_name):
        print('{} does not exist.'.format(FLAGS.database_file_name))
        sys.exit()

    graph_dict = {"group_id":"graph_id","vertex_id":"id","edge_src":"src_id","edge_dst":"dst_id"}
    gs = gSpan(
        database_file_name=FLAGS.database_file_name,
        min_support=FLAGS.min_support,
        min_num_vertices=FLAGS.lower_bound_of_num_vertices,
        max_num_vertices=FLAGS.upper_bound_of_num_vertices,
        max_ngraphs=FLAGS.num_graphs,
        is_undirected=(not FLAGS.directed),
        verbose=FLAGS.verbose,
        visualize=FLAGS.plot,
        where=FLAGS.where,
        graph_dict = graph_dict
    )

    gs.run()
    gs.time_stats()

    
    return gs


if __name__ == '__main__':
    args_str = '--min_support 4  --directed True --lower_bound_of_num_vertices 5 --upper_bound_of_num_vertices 10 --plot True --where True data/'
    FLAGS, _ = parser.parse_known_args(args=args_str.split())
    main(FLAGS)


100%|██████████| 1/1 [00:00<00:00,  2.01it/s]


Read:	0.6 s
Mine:	0.66 s
Total:	1.26 s


1

In [8]:
%pylab inline
gs = main(FLAGS)

/Users/tangting/opt/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


Populating the interactive namespace from numpy and matplotlib


100%|██████████| 1/1 [00:00<00:00,  2.63it/s]


Read:	0.42 s
Mine:	0.6200000000000001 s
Total:	1.04 s


## plot graphs in database

In [26]:
for g in gs.graphs.values():
    g.plot()

KeyboardInterrupt: 